# FOS Basic Example

## Outline
* [**Introduction**](#Introduction)
* [**Import Modules**](#Import-modules)
* [**Setup**](#Setup)
* [**Train**](#Train)
* [**Next Steps**](#Next-Steps)


## Introduction
This sample notebook shows the minimum required steps to train a model using **FOS**. FOS (the Greek word for light) is a Python framework that assists in the development of state of the art machine learning models in PyTorch. The primary focus is on making it a repeatable process with an easy to use API and good support for getting the nessecary insights into your model.  

This notebook trains a convolutional neural network that comes out of the box with torchvision, namely `resnet18`. The model takes as input an image of 224x224 pixels and predicts which of the possible 1000 object types are present in the image. To find our mote about the network architecture, checkout this [paper](https://arxiv.org/pdf/1512.03385.pdf). 

Since the main purpose is to demonstrate the use of FOS, we don't bother to download the pretrained weights or real images and save some bandwith. 

## Import modules
*FOS* supports the standard PyTorch classes and functions for defining models, dataloaders, loss functions and optimizers. So the first few  import statements should look very familiar if you have worked with PyTorch before.

For this notebook there are 3 classes required that belong to the *Fos* module: `Supervisor`, `NotebookMeter` and `Trainer`. Their purpose is explained later on. 

In [1]:
import torch
import torch.nn.functional as F
from torchvision.models import resnet18 

# Import the FOS classes we'll use in this example
from fos import Workout
from fos.callbacks import NotebookMeter
from fos.metrics import BinaryAccuracy

## Setup
We first create an instance of the model we want to train, so in this case the resnet18 model. Throughout this example we refer to the instance of model as the `predictor` in order to be able to differentiate from the `Supervisor` we create later on.

After the predictor is instantiated, the optimizer and loss function are created. If you are familiar with PyTorch this should all be straight forward. We choose Adam as the optimizer since it performs well in many scenarios. But feel free to swap it out for any other optimizer. And as a loss function we choose the `binary cross entropy` as that fits the multi-class classification problem well.

And finally time to create some random dummy data that mimics an image of 224x224 pixels and the target: 

     X: 4x3x224x224 = 4 samples in a batch x 3 channels (RGB) x 224 pixels width x 224 pixels height
     
     Y: 4x1000      = 4 samples in a batch x 1000 possible classes 

In a real world scenario's this would typically be implemented as a PyTorch Dataloader. But for the purpose of this notebook a simple list of random tensors will do just fine.

In [2]:
model = resnet18().to("cuda")
optim = torch.optim.Adam(model.parameters())
loss  = F.binary_cross_entropy_with_logits

data = [(torch.randn(4,3,224,224), torch.rand(4,1000).round()) for i in range(15)]

Now everything is ready to create the three nessecary **FOS**  objects:

1. A **Supervisor** that adds a loss function to the predictor that you want to train. The Supervisor instance is used by the trainer to perform the actual updating of the model. Throughout the examples we call this instance `model`. Under the hood, the Supervisor is still inheriting from `nn.Module`, so can be used were you would also any other PyTorch model. 

2. A **Meter** that handles the generated metrics like loss and custom metrics, although in this example there are no custom metrics defined. Here we use the NotebookMeter that will print the following info in a Jupyter notebook:
    * the global epoch and step counter
    * the metrics (in this case just the training loss)
    * the progress (percentage and time) per epoch
              
              
3. The **Trainer** that glues everything together and performs the training. When creating the trainer object, you need to pass as arguments a `supervisor`, an `optimizer` and a `meter`.

And that is really all that is required. So the minimum is three lines of **FOS** specific code:


In [3]:
workout = Workout(model, loss, optim, acc=BinaryAccuracy())
meter   = NotebookMeter(["loss", "acc"])

## Train
With everything being setup, the training can now start. In this case we run the training for 5 epochs. The trainer keeps track of the state of the training, so you can rerun the cell below multiple times and it will continue where it left off. This is ideal when you are working interactively in Notebooks.

We use random data, so you can safely ignore the loss values that are being printed. But just in case you are curious, since we only have a small set of training data the loss will go down quickly (clearly a case of overfitting). 

In [5]:
workout.fit(data, epochs=5, callbacks=meter)

[  5:    75]  - loss : 0.65553  - acc : 0.61750 : 100%|██████████|00:23<00:00
[  6:    90]  - loss : 0.64836  - acc : 0.61825 : 100%|██████████|00:00<00:00
[  7:   105]  - loss : 0.61911  - acc : 0.65250 : 100%|██████████|00:00<00:00
[  8:   120]  - loss : 0.59065  - acc : 0.67750 : 100%|██████████|00:00<00:00
[  9:   135]  - loss : 0.55449  - acc : 0.72050 : 100%|██████████|00:00<00:00
[ 10:   150]  - loss : 0.51287  - acc : 0.75075 : 100%|██████████|00:00<00:00
[ 11:   165]  - loss : 0.45823  - acc : 0.78425 : 100%|██████████|00:00<00:00
[ 12:   180]  - loss : 0.40269  - acc : 0.83650 : 100%|██████████|00:00<00:00
[ 13:   195]  - loss : 0.34828  - acc : 0.86825 : 100%|██████████|00:00<00:00
[ 14:   210]  - loss : 0.30029  - acc : 0.90200 : 100%|██████████|00:00<00:00
[ 15:   225]  - loss : 0.25655  - acc : 0.93150 : 100%|██████████|00:00<00:00
[ 16:   240]  - loss : 0.21905  - acc : 0.95275 : 100%|██████████|00:00<00:00
[ 17:   255]  - loss : 0.18743  - acc : 0.96750 : 100%|█████████

## Next Steps

We now have a trained model and that is all for this notebook. But there are other notebooks available that dive into much more details and there is also the API documentation itself. Please visit [github](https://github.com/quanta_ai/fos) to find out more.
